# Generating pitch tracks for the Guše visualizer

In [ ]:
import json
import os
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
folderName = '../files/pitchTracks/'
mbid = '0430401f-6ca3-404c-b4e2-458664ee2b33'
fileName = mbid + '.f0_fil.csv'

dur = 71

shahed = 1045.32

filePath = os.path.join(folderName, fileName)
print(filePath)

### From Hz to cents

In [ ]:
def h2c(t, h):
    return np.rint(1200 * np.log2(h/t))

In [ ]:
# Load csv
pitchTrackHz = np.genfromtxt(filePath, delimiter=",")
pitchTrackHz

In [ ]:
maxCent = h2c(shahed, np.max(pitchTrackHz[:,1]))
print("Max:", maxCent)

minCent = h2c(shahed, np.min(pitchTrackHz[:,1]))
print("Min:", minCent)

In [ ]:
# Prepare dic
pitchTrackDic = {}

for frame in pitchTrackHz:
    pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]

In [ ]:
# Find duration

if pitchTrackHz[-1,0] > dur:
    dur = pitchTrackHz[-1,0]

In [ ]:
# Cent pitch track

pitchTrackCents = {}

for i in range(dur * 100):
    k = "{:.2f}".format(i/100)
    if k not in pitchTrackDic.keys():
        pitchTrackCents[k] = "s"
    else:
        pitchTrackCents[k] = h2c(shahed, pitchTrackDic[k])

pitchTrackCents

In [ ]:
with open(os.path.join(folderName, mbid + "_pitchTrack.json"), 'w') as f:
    json.dump(pitchTrackCents, f)

### Pitch histogram

In [12]:
import intonation
import numpy as np
import json
import os

In [8]:
def pitch2cents(pitch, ref):
    '''
    Converts a pitch track in Hz to cents given a reference pitch
    
    Args:
        pitch (np.array): a numpy array with two columns, first for time,
                          second for pitch
        ref (float): the reference pitch
    
    Returns:
        cents (np.array) a numpy array with one column of pitch in cents
    '''

    return 1200 * np.log2(pitch[:,1] / float(ref))

In [13]:
pitchTrackFolder = '../files/pitchTracks'
mbid = '0430401f-6ca3-404c-b4e2-458664ee2b33'
pitchTrackExtension = '.f0_fil.csv'

shahed = 1045.32

In [18]:
pitchTrack = np.genfromtxt(os.path.join(pitchTrackFolder, mbid + pitchTrackExtension), delimiter=',')
centsTrack = pitch2cents(pitchTrack, shahed)
# filteredTrack = centsTrack[np.logical_and(centsTrack >= lowCut, centsTrack <= highCut)]
# pitch_obj = intonation.Pitch(np.arange(len(filteredTrack)), filteredTrack)
pitch_obj = intonation.Pitch(np.arange(len(centsTrack)), centsTrack)
rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist()

print(rec_obj.histogram.x)
print(rec_obj.histogram.y)

[-721.90905674 -720.90905301 -719.90904928 ...  545.09566834  546.09567207
  547.0956758 ]
[3.57072261e-05 3.51922211e-05 3.41870439e-05 ... 3.73267216e-05
 3.74255304e-05 3.74917954e-05]


In [34]:
histDic = {}

for i in range(rec_obj.histogram.x.size):
    # pos = '{:.2f}'.format(rec_obj.histogram.x[i])
    x = round(rec_obj.histogram.x[i], 2)
    y = round(rec_obj.histogram.y[i] * 10000, 2)
    histDic[x] = y

histMin = np.min(list(histDic.values()))
histMax = np.max(list(histDic.values()))

print(histMin, histMax)

histList = []
for k, v in histDic.items():
    histList.append([k,v])

hist2json = {'min': histMin, 'max': histMax, 'hist': histList}
hist2json

0.0 130.9


{'min': 0.0,
 'max': 130.9,
 'hist': [[-721.91, 0.36],
  [-720.91, 0.35],
  [-719.91, 0.34],
  [-718.91, 0.33],
  [-717.91, 0.31],
  [-716.91, 0.29],
  [-715.91, 0.27],
  [-714.91, 0.24],
  [-713.91, 0.22],
  [-712.91, 0.19],
  [-711.91, 0.17],
  [-710.91, 0.15],
  [-709.91, 0.14],
  [-708.91, 0.13],
  [-707.91, 0.12],
  [-706.91, 0.11],
  [-705.91, 0.11],
  [-704.91, 0.11],
  [-703.91, 0.11],
  [-702.91, 0.11],
  [-701.91, 0.11],
  [-700.91, 0.12],
  [-699.91, 0.12],
  [-698.91, 0.12],
  [-697.91, 0.12],
  [-696.91, 0.13],
  [-695.91, 0.13],
  [-694.91, 0.14],
  [-693.91, 0.15],
  [-692.91, 0.16],
  [-691.91, 0.17],
  [-690.91, 0.19],
  [-689.91, 0.21],
  [-688.91, 0.24],
  [-687.91, 0.27],
  [-686.91, 0.3],
  [-685.91, 0.34],
  [-684.91, 0.37],
  [-683.91, 0.41],
  [-682.91, 0.44],
  [-681.91, 0.47],
  [-680.91, 0.5],
  [-679.91, 0.52],
  [-678.91, 0.53],
  [-677.91, 0.54],
  [-676.91, 0.54],
  [-675.91, 0.53],
  [-674.91, 0.51],
  [-673.91, 0.48],
  [-672.91, 0.45],
  [-671.91, 0.42

In [36]:
pitchHistogramFolder = '../files/pitchHistograms'
pitchHistogramExtension = '_pitchHistogram.json'

json.dumps(hist2json)

with open(os.path.join(pitchHistogramFolder, mbid + pitchHistogramExtension), 'w') as f:
    json.dump(hist2json, f)